In [ ]:
import os
from zen_garden.postprocess.results import Results

In [ ]:
dataset_name_1 = 'Data_WT'
output_path_1 = os.path.join("outputs", dataset_name_1)
r1 = Results(output_path_1)

In [ ]:
dataset_name_2 = 'Data_WT_new'
output_path_2 = os.path.join("outputs", dataset_name_2)
r2 = Results(output_path_2)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Reset and clean up
demand1 = r1.get_full_ts("demand").reset_index()
demand2 = r2.get_full_ts("demand").reset_index()

# Rename scenario column for clarity
demand1 = demand1.rename(columns={"level_0": "scenario"})
demand2 = demand2.rename(columns={"level_0": "scenario"})

# Define time step columns (float or int)
time_cols = [col for col in demand1.columns if isinstance(col, (int, float))]
times = time_cols  # use directly without adjustment

# Loop over scenarios
scenarios = demand1["scenario"].unique()

for scenario in scenarios:
    d1_s = demand1[demand1["scenario"] == scenario]
    d2_s = demand2[demand2["scenario"] == scenario]

    # --- Define times for this scenario ---
    time_cols = [col for col in d1_s.columns if isinstance(col, (int, float))]
    times = time_cols  # use directly

    # --- Group by carrier ---
    d1_grouped = d1_s.groupby("carrier")[times].sum()
    d2_grouped = d2_s.groupby("carrier")[times].sum()

    carriers = sorted(set(d1_grouped.index).union(d2_grouped.index))
    non_zero = [
        carrier for carrier in carriers
        if d1_grouped.loc[carrier].sum() > 0 or d2_grouped.loc[carrier].sum() > 0
    ]

    # --- Plot per carrier ---
    for carrier in non_zero:
        plt.figure(figsize=(10, 5))
        plt.plot(times, d1_grouped.loc[carrier], label=f"{dataset_name_1}", linestyle="--")
        plt.plot(times, d2_grouped.loc[carrier], label=f"{dataset_name_2}", linestyle="-")
        plt.title(f"Total Demand — Scenario: {scenario} — Carrier: {carrier}")
        plt.xlabel("Year")
        plt.ylabel("Demand [GWh]")
        plt.grid(True, linestyle="--")
        plt.legend()
        plt.tight_layout()

        # --- Custom xticks every 5 time steps (for yearly labeling) ---
        step_interval = 5  # assuming 5 steps per year
        xticks_positions = times[::step_interval]
        xticks_labels = [str(2021 + i) for i in range(len(xticks_positions))]  # start year = 2021
        plt.xticks(ticks=xticks_positions, labels=xticks_labels)

        plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Load cost_total data from both datasets
cost1 = r1.get_full_ts("cost_total").reset_index()
cost2 = r2.get_full_ts("cost_total").reset_index()

# Rename the first column to 'scenario'
cost1.columns.values[0] = "scenario"
cost2.columns.values[0] = "scenario"

# Get time step columns (e.g., 0, 1, 2, ...)
time_cols = [col for col in cost1.columns if isinstance(col, (int, float))]
# Generate actual years (assuming 2021 start, adjust if needed)
times = [2021 + i for i in range(len(time_cols))]

# Get unique scenarios
scenarios = cost1["scenario"].unique()

# Loop over each scenario
for scenario in scenarios:
    c1_row = cost1[cost1["scenario"] == scenario]
    c2_row = cost2[cost2["scenario"] == scenario]

    if c1_row.empty or c2_row.empty:
        print(f"Skipping missing scenario: {scenario}")
        continue

    # Extract cost values
    c1_values = c1_row.iloc[0][time_cols].values
    c2_values = c2_row.iloc[0][time_cols].values

    # Plot
    plt.figure(figsize=(10, 5))
    plt.plot(times, c1_values, label="Data_Wind", linestyle="--")
    plt.plot(times, c2_values, label="Data_Wind_new", linestyle="-")
    plt.title(f"Total System Cost — Scenario: {scenario}")
    plt.xlabel("Year")
    plt.ylabel("Cost [M€]")
    plt.grid(True, linestyle="--")
    plt.legend()
    plt.tight_layout()

    # Format Y-axis with thousand separators
    ax = plt.gca()
    ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{x:,.0f}'))

    plt.show()
